# PDMS 3D model contextualization
This notebook contains a workflow for contextualizing 3D models made in PDMS which typically works well for oil and gas 3D models. 

Authors: Alina Astrakova and Anders Hafreager

In [ ]:
import pandas as pd

from getpass import getpass
from cognite.experimental import CogniteClient
from cognite.client.data_classes.three_d import ThreeDAssetMapping

# Initialize

In [ ]:
client = CogniteClient(
    api_key=getpass(), 
    project="publicdata", 
    client_name="dshub"
)
client.login.status()

In [ ]:
# Find the 3D model you want to contextualize
client.three_d.models.list(limit=-1)

In [ ]:
# For a given model (and its model id), find the 3D model revision you want to contextualize
# If you don't find the revision, try published=False
client.three_d.revisions.list(model_id=4715379429968321, published=True) 

In [ ]:
# List all root assets to find the root asset you want to map to
client.assets.list(root=True, limit=-1)

In [ ]:
# Define 3d model_id and revision
model_id = 4715379429968321
revision_id = 5688854005909501

# Define root_id for assets
root_id = 6687602007296940

# Download data

In [ ]:
# Download 3D nodes. This may take a while ...
threed_nodes = client.three_d.revisions.list_nodes(model_id=model_id, revision_id=revision_id, limit=-1)

In [ ]:
# The 3D node hierarchy is often made of nodes with names on the form "/21PT1019"
# with children nodes with names "BRANCH 1 of /21PT1019".
# We only want to map the parent node, so remove all nodes with such names.
nodes_list = threed_nodes.dump()
filtered_nodes = list(filter(lambda x: x["name"] != "", nodes_list))
print("%d non empty node names" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "EQUIPMENT" not in x["name"], filtered_nodes))
print("%d node names without EQUIPMENT" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "BRANCH" not in x["name"], filtered_nodes))
print("%d node names without BRANCH" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "STRUCTURE" not in x["name"], filtered_nodes))
print("%d node names without STRUCTURE" % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: " OF " not in x["name"], filtered_nodes))
print("%d node names without OF " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: " of " not in x["name"], filtered_nodes))
print("%d node names without of " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Box" not in x["name"], filtered_nodes))
print("%d node names without Box " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Cylinder" not in x["name"], filtered_nodes))
print("%d node names without Cylinder " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Facet Group" not in x["name"], filtered_nodes))
print("%d node names without Facet Group " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "curve" not in x["name"], filtered_nodes))
print("%d node names without curve " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Pyramid" not in x["name"], filtered_nodes))
print("%d node names without Pyramid " % len(filtered_nodes))
filtered_nodes = list(filter(lambda x: "Line" not in x["name"], filtered_nodes))
print("%d node names without Line " % len(filtered_nodes))
filtered_node_names = list(map(lambda x: {"name": x["name"], "id": x["id"]}, filtered_nodes))

In [ ]:
# Download assets
assets = client.assets.retrieve_subtree(root_id)
asset_names_ids = [{"name": x.name, "id": x.id} for x in assets]
print(f"Downloaded {len(asset_names_ids)} assets")

# Run the entity matcher

In [ ]:
# Create an entity matching model
model = client.entity_matching.fit(match_from=filtered_node_names, match_to=asset_names_ids)

In [ ]:
# Get predictions from model
job = model.predict()

In [ ]:
# Check predictions job status. The job will take a few minutes to complete
print(f"Job is {job.update_status()}, last updated at {job.status_timestamp}")

In [ ]:
# When the job is finished, get the results
matches = job.result["items"]

In [ ]:
# This may require some work of verification. Set a threshold and run 
threshold = 0.9
good_matches = [m for m in matches if (len(m["matches"]) > 0 and m["matches"][0]["score"] > threshold)]
print("Got %d matches with score > %f" % (len(good_matches), threshold))
pd.DataFrame.from_records([(m["matchFrom"]["name"], m["matches"][0]["matchTo"]["name"], m["matches"][0]["score"]) for m in good_matches])

# Create asset mappings

In [ ]:
asset_mappings = []
for match in good_matches:
    asset_id = match["matches"][0]["matchTo"]["id"]
    node_id = match["matchFrom"]["id"]
    asset_mappings.append(ThreeDAssetMapping(node_id=node_id, asset_id=asset_id))

In [ ]:
# Write mappings to CDF
res = client.three_d.asset_mappings.create(model_id=model_id, revision_id=revision_id, asset_mapping=asset_mappings)

# Delete asset mappings

In [ ]:
# Uncomment the last line to delete all existing mappings (if you want to create new ones)
# WARNING: This cannot be undone
res = client.three_d.asset_mappings.list(model_id=model_id, revision_id=revision_id, limit=-1)
existing_mappings = list(map(lambda x: ThreeDAssetMapping(node_id=x.node_id, asset_id=x.asset_id), res))
#res = client.three_d.asset_mappings.delete(model_id=model_id, revision_id=revision_id, asset_mapping=existing_mappings)